<a href="https://colab.research.google.com/github/p-s-nayak/DL-NeuralNetwork-HUB/blob/main/02_pyt_In_Place_vs_Out_of_Place_Operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`Out-of-place operation`
- Creates a new tensor with the result and leaves the original tensor unchanged.
- → Safer, predictable, preserves history for autograd.

`In-place operation`
- Modifies the existing tensor directly (mutates its data in memory) without allocating a new tensor.
- → Saves memory and can be slightly faster in memory-bound cases, but riskier.

# 🔍 How to Recognize In-Place vs Out-of-Place Operations (PyTorch)

> **Rule of thumb:**  
If an operation **ends with `_` or uses assignment (`+=`, slicing)**, it is **in-place**.

---

## Recognition Table

| Type | Naming Convention | Examples (Common) |
|----|------------------|------------------|
| **Out-of-Place** | Normal function name | `torch.add(x, y)`, `x + y`, `torch.relu(x)`, `torch.mul(x, 2)`, `torch.clamp(x, 0, 1)` |
| **In-Place (API)** | Ends with underscore `_` | `x.add_(y)`, `x.mul_(2)`, `x.clamp_(0, 1)`, `x.relu_()`, `x.zero_()` |
| **In-Place (Python syntax)** | Assignment operators / slicing | `x += y`, `x -= y`, `x *= y`, `x /= y`, `x **= 2`, `x[0] = 5`, `x[:, 0] = 0` |


## Quick Memory Trick

> **Underscore or assignment = mutation**  
> **Expression without assignment = new tensor**



## Why This Matters

- In-place ops modify the original tensor
- Can break `autograd` if gradients are required
- Can silently affect other tensors sharing memory (views)

Use in-place operations **only when mutation is intentional**.


In [ ]:
import torch

x = torch.tensor([1., 2., 3.])

# Out-of-place (creates new tensor)
y_out = x + 10          # or torch.add(x, 10)
print(y_out)            # tensor([11., 12., 13.])
print(x)                # still [1., 2., 3.]  ← unchanged

# In-place (modifies x directly)
x.add_(10)              # or x += 10
print(x)                # tensor([11., 12., 13.])  ← x is changed!

tensor([11., 12., 13.])
tensor([1., 2., 3.])
tensor([11., 12., 13.])


# ⚖️ When to Use Which: In-Place vs Out-of-Place Operations

Choosing between **in-place** and **out-of-place** operations affects **correctness, memory usage, and gradient computation**.


## ✅ Use Out-of-Place Operations When

- You need to **preserve the original tensor**
- The operation is part of the **computational graph**
- Gradients must be computed **safely and correctly**
- You want **clear, debuggable code**

**Typical scenarios**
- Model training
- Loss computation
- Intermediate activations


## ⚠️ Use In-Place Operations When

- **Memory constraints** are critical
- You are **certain the original tensor is no longer needed**
- You fully understand the **autograd implications**
- Working in:
  - inference pipelines
  - `torch.no_grad()` blocks
  - post-processing steps


## 🚨 Caution (Important)

> **In-place operations can break backpropagation**

Be extra careful when:
- The tensor has `requires_grad=True`
- The tensor is used in multiple branches
- The tensor is a **view** of another tensor

In-place modification may:
- overwrite values needed for gradient computation
- cause runtime errors during `backward()`
- silently produce incorrect gradients


## 🧠 Final Rule of Thumb

> **Training → Out-of-place**  
> **Inference / memory optimization → In-place (with care)**

